# Finding the LRP signal with mne_pipeline

The `mne_pipeline` class contains a collection of helper and wrapper methods
for functions for the `mne` module.
Here we will take a look at how you would go about finding the
[lateralized readiness potential (LRP)](https://en.wikipedia.org/wiki/Lateralized_readiness_potential)
signal using the `mne_pipeline` class.

The data we will be using stems from an experiment where a participant had to choose
between two options of dividing money up between themselves and another anonymous
participant (i.e. a forced-choice version of the
[dictator game](https://en.wikipedia.org/wiki/Dictator_game)) by pressing a button
with their left index finger for the left option and with their right index finger
for the right option.
We therefore expect to see an LRP build up before either of the buttons is pressed.


## Importing and Preparing the Data

First we will have to import the modules we need.
Make sure `mne_pipeline.py` is within the same directory of the script or
otherwise provide the relative path during the import.

In [3]:
# TODO: Check which imports are really needed
import os
import mne
import pandas as pd
import numpy as np
import mne_pipeline

We now _import the data_ of a participant.
In a proper analysis script this might be done in a loop over all participants so that you can
deal with them one by one.

In [ ]:
triggers = {'Masked': 2, 'Reveal': 4, 'Left_choice': 8, 'Right_choice': 16, 'No_choice': 32}

eeg_prep = mne_pipeline.EEGPrep(
    eeg_path=os.path.join('Data', 'raw', 'kevin_raw.fif'),
    trigger_dict=triggers,
    participant_identifier='test_participant')

As you can see, the object `eeg_prep` is an instance of the EEGPrep class.
To initialize it we give it the path to our EEG file (which can be either in .bdf or .fif format),
the trigger dictionary and an identifier for the participant (which can also be a number).
The trigger dictionary contains the labels of the triggers as keys and the corresponding
value of the trigger channel as values. In this case `'masked'` means that at that time
the stimulus was presented but the numbers were not visible yet.
At this time the trigger channel (called `Status` or `Stim`) takes on the value of 2.

Now we have our `eeg_prep` object which has a bunch of attributes
(for example the raw data itself or the `participant_id` we just gave it) and methods.
The methods can either have a return value so that it will be used as
`new_value = eeg_prep.some_method()` or it can operate on the attributes within the object
 (again, for example the raw data) directly.

 The next two methods we are going to apply do exactly that.
 The `fix_channels` method removes the superfluous `1-` at the start of the channel names
 and _sets the montage_, which is what tells `mne` where on the head each electrode was attached.
 Then `set_references` automatically _rereferences your signal_ to the average of the
 mastoid reference electrodes (the ones behind the ear) and creates new channels
 from the bipolar electrodes like ones capturing eye and hand movement.

In [ ]:
eeg_prep.fix_channels(n_ext_channels=9,
                      ext_ch_mapping=None)

eeg_prep.set_references(bipolar_dict=dict(eye_horizontal=['PO10', 'FT10'],
                                          eye_vertical=['HeRe', 'FT10'],
                                          right_hand=['HeLi', 'VeDo'],
                                          left_hand=['VeUp', 'EMG1a']))

As you can see, `fix_channels` also gives you the option to set the number of extra channels
(i.e. everything that is not included in the 64 electrode setup) and lets you provide your own
type mapping for those extra channels if desired (see the documentation in the code for details).

`set_references` takes a dictionary for the bipolar electrodes called `bipolar_dict`.
Here the dictionary keys will be used as the channel names for the newly generated channels and the
dictionary values should be a list containing the anode and cathode of the bipolar channel
(e.g. the electrodes above and below the eye for the `eye_vertical` channel).
If you use a reference channel other than the average of `PO9` and `FT9`, you can also
use the `ref_ch` argument (not used here) to provide the channels to use as reference as a list.

Now that MNE and python know how to handle our data we want to _retrieve the events_ we recorded during our study.
Our `eeg_prep` object already contains the `triggers` dictionary we provided when we created it.
The method `find_events` is basically a wrapper for `mne.find_events` and saves its result in an attribute called
`events`.
All arguments are passed on to `mne.find_events` so if you are struggeling you can read
[its documentation](https://mne.tools/stable/generated/mne.find_events.html).

In [ ]:
eeg_prep.find_events(stim_channel='Status', consecutive=True, min_duration=.01)

What we need here is to tell the method which channel contains the trigger information with the `stim_channel`
argument. Set `consecutive` to `True` if your trigger channel does not return to zero after each trigger.
`min_duration` tells MNE that any fluctuation in the signal smaller than that value in seconds is just noise and
should not be considered a trigger signal.

__Hint:__ _If you are programming an experiment, make sure to either always set the trigger channel back to zero
after each event or make sure that the same trigger values are never used twice in a row.
Setting the channel to 8 for example and then setting it to 8 again for the second event will just produce a flat line
from which the timing of the second event cannot be recovered._


## Cleaning the Data

In [ ]:
eeg_prep.find_ica_components()
eeg_prep.remove_ica_components()

Here we start by applying an independent component analysis (ICA) to filter out eye- and other movement artifacts.
Running the `find_ica_components` method will fit the ICA on your data.
If you know that you have very noisy data between trials i.e. from people moving a lot, you can first create
epochs (see below) and then in `find_ica_components` set `fit_on_epochs=False`.
That way only the data within the epochs will be considered.
Further, if you know of bad episodes (e.g. a yawn), you can annotate the segment as 'bad' and it will be excluded
from the fitting procedure. Please consult the
[MNE documentation](https://mne.tools/stable/overview/index.html#documentation-overview) on how to do this.
You can always modify the `mne.raw` object as `eeg_prep.raw`.

Further options are `high_pass_freq` which applied a high-pass filter (i.e. filtering out low frequency noise)
before fitting the ICA, `decim` which speeds up the process by only using every `decim`th sample,
and `n_components` which is set to 20 by default and decides how many ICA components are extracted.
All other arguments are passed on to the `mne.ICA` method.

`find_ica_components` will show you a plot with the components it fit.
When excluding components you have different options to do so.
Should you wish to decide manually which components to exclude, make sure to create a file called
`participant_PARTICIPANTID_rejected_ICA_components.csv` and list the rejected components (one component per line)
in it. If you have mistakenly closed the plot just run `find_ica_components` again.
If you have changed nothing in the arguments it will not re-fit the ICA but show you the plot again.

After fitting the ICA we are able to actually remove the components we selected.
This is done by running `remove_ica_components`. If you have created a list of components you want to reject as
described above, you can set `reject_from_file=True`. In any case you should provide the path to the folder where
you either already store or want to store the files with the rejection lists with `reject_list_file_location`.
If you provide a path to a folder that does not exist yet, it will be created.
Any further arguments are passed on to `mne.ica.apply`.

If you did not set `reject_from_file=True` MNE's `find_bads_eog` method will be used to decide which components
to reject. The resulting list of components will then be written into the `[...]_rejected_ICA_components.csv` file.
Further, if bad components are detected, and informative plot will be drawn showing the eog scores of the components.

Next, we will apply a filter to our signal.

In [ ]:
eeg_prep.filters(low_freq=1/7, high_freq=128, notch_freq=50)

As you might imagine, this filters out frequencies below `low_freq` and above `high_freq`.
Further a notch filter is applied to `notch_freq` and its harmonies up to `high_freq` to get rid of electrical wire
noise.


## Creating Epochs and Saving

Next, we will create epochs from our raw data. This again is relatively straight forward.
You can either get the epochs as an attribute of the `eeg_prep` object or you can get it as a pandas data frame
so you can apply other methods outside of MNE to it.

In [ ]:
eeg_prep.get_epochs(event_labels=['Left_choice', 'Right_choice'],
                    tmin=-1.5, tmax=.2, baseline=(-1.5, -1.2))
data_frame = eeg_prep.get_epochs_df(event_labels=['Left_choice', 'Right_choice'],
                                    tmin=-1.5, tmax=.2, baseline=(-1.5, -1.2))

The `event_labels` are from the `trigger_dict` you provided at the start and contains all events around which you
want to build your epochs.
The arguments `tmin`, `tmax`, and `baseline` are passed on to `mne.Epochs`.
The first two describe how many seconds before and after the event the epoch should start and stop.
`baseline` should provide a time interval whose average voltage will be subtracted from the epoch so that slow
fluctuations don't increase the variance between epochs.

If you now want to save your prepared data to work with it later, you can do so with the `save_prepared_data` method.

In [ ]:
eeg_prep.save_prepared_data(save_path='prepared_data', overwrite=True)

This will create a new `.fif` file in the location you provided in `save_path`.
If you wish you could also set `save_events=True` and `save_epochs=True`.
Events will be saved using pythons `pickle` library and epochs are also used as `.fif` files.
Overwrite as well as any other named arguments are passed on to `raw.save`.